In [2]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 8.1 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 8.5 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 4.3 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

In [27]:
df_data_all = pd.read_csv("./KPIs.csv")

In [62]:
# Elimino las últimas tres columnas
df_data = df_data_all.iloc[ : , :-3]

In [64]:
arr_columns = df_data.iloc[0:0,:]
arr_columns

,CATEGORIA,TEMA,DATO BUSCADO,¿Se tiene?,ORIGEN,Campo agregado,Nombre del campo


## FILTRACIÓN POR LA COLUMNA CATEGORIA

In [69]:
df_data[df_data['CATEGORIA'] == 'TI']

,CATEGORIA,TEMA,DATO BUSCADO,¿Se tiene?,ORIGEN,Campo agregado,Nombre del campo
64,TI,Equipos de computo,Equipo total de equipos de cómputo por área,No,"Facturación, Inventario y Adquisicón",No,NaN
65,TI,Equipos de computo,Equipo total de equipos de cómputo,No,"Facturación, Inventario y Adquisicón",No,NaN
66,TI,Equipos de computo,Ver el total de equipos de computo por cada sala.,No,"Facturación, Inventario y Adquisicón",No,NaN
67,TI,Equipos de computo,Ver el total de equipos de computo por cada se...,No,"Facturación, Inventario y Adquisicón",No,NaN
68,TI,Equipos de computo,Ver el total de cómputo por cada dirección,No,"Facturación, Inventario y Adquisicón",No,NaN
69,TI,Equipos de computo,Ver el porcentaje de los diferentes tipos de c...,No,"Facturación, Inventario y Adquisicón",No,NaN
70,TI,Equipos de computo,Ver el total de los diferentes tipos de cómputo,No,"Facturación, Inventario y Adquisicón",No,NaN
71,TI,Equipos de computo,Ver el porcentaje de los equipos de cómputo ba...,No,"Facturación, Inventario y Adquisicón",No,NaN
72,TI,Equipos de computo,Ver el total de los equipos de cómputo bajo ar...,No,"Facturación, Inventario y Adquisicón",No,NaN
